# Projekt: Wärmeleitung

### Initialisierung

In [5]:
import numpy as np
import scipy
from scipy.sparse import lil_matrix
from scipy.sparse import coo_matrix
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve
factorial = np.vectorize(np.math.factorial)

#funktion: factorial, np.vectorize

## Aufgabe 1

In [28]:
n = 3 #Anzahl Elemente in 1
n_dach = 1 #Anzahl der zusätzlichen Auswertungspunkte je Element in 1
n_schlange = 7 #Ordnung der Quadratur in 1
n_p = 100 #Anzahl der Zeitschritte in 1
beta = 1/4 # Newmark-Koeffizient in 1
gamma = 1/2 #Newmark-Koeffizient in 1
eta = 0.1 # Zeitschrittweite in s
l = 1 #Länge des Balken in m
µ = 1 #Längenspezifsche Masse in kg/m
E = 1 #Elastizitätsmodul in N/m²
I = 1 #Flächenträgheitsmoment in m^4
q = 1 #Streckenast in N/m

B = np.array([[0,1,0],[0,2,0],[n,3,0],[n,4,0]])

## Aufgabe 2

#### keine Loops und ifs, kein sparse Format, erzeugen von nützlichen Array mit verschiedenen Indizes. Schreibe eine Funktion getindizes

#### a) 

In [61]:
def getindex_a(n):
    m = n+1
    m1 = np.arange(0,m)
    m2 = np.arange(0,n)
    m3 = np.arange(0,m)
    M1, M2, M3= np.meshgrid(m1,m2,m3)
    M4 = np.zeros((n,m,m)) 
    #1. anzahl matrizen, 2. anzahl spalten, 3. anzahl reihen
    M5 = np.zeros((n,m,m)) 
    M4 = 2*M2+M1
    M5 = 2*M2+M3
    return(M2,M1,M3,M4,M5)

In [63]:
getindex_a(n)

In [110]:
def getindex_b(n):
    m = n+1
    m1 = np.arange(0,m) #anzhal zeilen
    m2 = np.arange(0,n) #anzahl matrizen
    m3 = [0] #anzahl spalten
    M1, M2,M3 = np.meshgrid(m1,m2,m3)
    M4 = np.zeros((n,m,1)) #((marizen,zeilen,spalten))
    M4 = 2*M2+M1
    return(M2,M1,M4)

In [113]:
getindex_b(n);

## Aufgabe 3

#### einfach abschreiben Funktionen getMbar, getSbar, getqbar

In [24]:
def getMbar(µ,h):
    M_ = (µ*h/420)*np.array(
        [[156, 22*h,54,(-13*h)],
        [(22*h),(4*h**2),(13*h),(-3*h**2)],
        [54,(13*h),156,(-22*h)],
         [(-13*h),(-3*h**2),(-22*h),(4*h**2)]])
    return(M_)

In [160]:
def getSbar(E,I,h):
    S_ = (E*I/h**3)*np.array(
    [[12,(6*h),(-12),(6*h)],
    [(6*h),(4*h**2),(-6*h),(2*h**2)],
     [(-12),(-6*h),12,(-6*h)],
    [(6*h),(2*h**2),(-6*h),(4*h**2)]])
    return(S_)

In [82]:
def getqbar(q,h):
    q_ = (q*h/12)*np.array([[6],
                           [h],
                           [6],
                           [-h]])
    return(q_)

## Aufgabe 4

#### Funktionen so anpassen, das sie Elementmatrizen in Form eine 3D-Arrays ausgibt.

a)

In [25]:
def getMbar_mod(µ,h,n):
    M_ = getMbar(µ,h)
    m = n+1
    M_l = np.zeros((n,m,m))
    M_l[:] = M_
    return(M_l)

#### 
b)

In [161]:
def getSbar_mod(E,I,h,n):
    m = n+1
    S_ = getSbar(E,I,n)
    S_l = np.zeros((n,m,m))
    S_l[:] = S_
    return(S_l)
#getSbar_mod(E,I,1,n)

#### 
c)

In [165]:
def getqbar_mod(q,h):
    m = n+1
    q_l = np.zeros((n,m,1))
    q_ = getqbar(q,h)
    q_l[:] = q_
    return(q_l)

## Aufgabe 5

#### ab hier sparse und scipy und NICHT mehr numpy!

## Fragen: 
- aus Elementmatrix der vorherigen Aufgabe, warum brauche ich die? Warum kann ich nicht die einfache M nehmen?

In [19]:
import warnings

# Ignore all warnings globally, scipy.arange gibts irgendwann nicht mehr und wir sollen np verwenden aber tutor sagt nein
warnings.filterwarnings("ignore")

#### 
a)

In [98]:
def getM(n):
    m = n+1
    A = lil_matrix(2*m, 2*m)
    i = scipy.arange(0,2*m)
    j = scipy.arange(0,2*m)
    i_ = scipy.arange(0,m)
    j_ = scipy.arange(0,m)
    M = lil_matrix(m,m)
    M = getMbar(µ,h = 1)
    L = scipy.arange(0,n)
    for l in L:
        for val in i_:
            i = 2*l + val
            for val_ in j_:
                j = 2*l + val_
                A[i, j] = A[i,j] + M[val,val_]
                A = A.tocsr()
    return(A)

#### 

b)


In [99]:
def getS(n):
    m = n+1
    A = lil_matrix(2*m, 2*m)
    i = scipy.arange(0,2*m)
    j = scipy.arange(0,2*m)
    i_ = scipy.arange(0,m)
    j_ = scipy.arange(0,m)
    M = lil_matrix(m,m)
    S = getSbar(E,I,n)
    L = scipy.arange(0,n)
    for l in L:
        for val in i_:
            i = 2*l + val
            for val_ in j_:
                j = 2*l + val_
                A[i, j] = A[i,j] + S[val,val_]
                A = A.tocsr()

    return(A)

#### 

c)

In [101]:
def getvq(n):
    m = n+1
    A = lil_matrix((2*m,1)) #muss das scipy.sparse.coo_matrix sein oder kann das auch lil_matrix sein?
    j = scipy.arange(2*m,1)
    i = 0
    j_ = scipy.arange(0,m)
    M = lil_matrix(m,1)
    M = getqbar(q,h = 1)
    L = scipy.arange(0,n)
    for l in L:
        for val in j_:
            j = 2*l + val
            A[j, 0] = A[j,0] + M[val,0]
            A = A.tocsr()

    return(A)

In [102]:
print(getvq(3))

  (0, 0)	0.5
  (1, 0)	0.08333333333333333
  (2, 0)	1.0
  (3, 0)	0.0
  (4, 0)	1.0
  (5, 0)	0.0
  (6, 0)	0.5
  (7, 0)	-0.08333333333333333


## Aufgabe 6

#### get C für Nebenbedingungen

In [7]:
#z.B.
n = 3
m = n+1
a_k = b_k = M_k = Q_k = 0
B = [[0,1,a_k],
    [0,2,b_k],
    [n,3,M_k],
    [n,4,Q_k]]

In [27]:
#gegeben: n und B
#durch B durchitterieren und Einträge mit 1 bzw. 2 finden:
#noch bisschen hässlich glaube ich, viel Schleifenzeug
j = l = 0
for i in range(len(B)):
    if B[i][1] == 1:
        j = j+1
    if B[i][1] == 2:
        l = l+1
        
E_1 = lil_matrix((2*m,j))
E_2 = lil_matrix((2*m,l))

j = l = 0
for i in range(len(B)):
    k = B[i][0]
    if B[i][1] == 1:
        if j == 0:
            E_1[2*k] = 1
        if j >0:
            E_1[2*k][j] = 1
        j = j+1
    if B[i][1] == 2:
        if l == 0:
            E_2[(2*k+1)] = 1
        if l >0:
            E_2[(2*k+1)][l] = 1
        l = l+1
print(E_2)

  (1, 0)	1.0
